In [2]:
#importing all liberaries
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
#fetching the page for processing
URL = "http://lhr.nu.edu.pk/faculty/"
soup = BeautifulSoup(requests.get(URL).content,'html.parser')

In [ ]:
dept = ['fsc', 'ee', 'cv', 'fsm', 'ss']

In [3]:
#   -   -   -   -   -   -   Step-1   -   -   -   -   -   -   -

#array containing list of all teachers of all the departments
faculty = [] 
[faculty.append(soup.find('div',id=dp).findAll('div',class_='card')) for dp in dept]

NU = 'http://lhr.nu.edu.pk/'

#function used to make dataframe of all departmants
def dataframe_maker_1(faculty):
    ID, facultyURL, email, name, image, designation, HECApproved = [], [], [], [], [], [], []

    for f in faculty:
        ID.append(f.find('a',class_='faculty-link')['href'].split('/')[-1])
        facultyURL.append(NU+f.find('a',class_='faculty-link')['href'])
        email.append(f.find('p',class_='mb-0').text.strip())
        name.append(f.find('h5',class_='text-center').text.strip())
        designation.append(f.find('p',class_='small').text.strip().split('\n')[0])
        HECApproved.append((f.find('p',class_='small').text.find('HEC Approved PhD Supervisor') >-1))
        image.append(NU+f.find('img',class_='card-img-top')['src'])

    return pd.DataFrame({
        'ID':ID,
        'name':name,
        'designation':designation,
        'HEC Approved PhD Supervisor': HECApproved,
        'email':email,
        'facultyURL':facultyURL,
        'image':image
    })

#saving the dataframes of all departments as csv
[dataframe_maker_1(faculty[i]).to_csv(dept[i]+'.csv') for i in range(len(dept))]


[None, None, None, None, None]

In [4]:
#   -   -   -   -   -   -   Step-2   -   -   -   -   -   -   -

samples, ids_list = [], []

#saving all sample data in an array 
[samples.append(pd.read_csv(dept[i]+'.csv').sample(frac=0.1)) for i in range(len(dept))]

#saving ids of all sample data in an array 
[ids_list.append(s['ID']) for s in samples]

#function used to make dataframe of all departmants
def dataframe_maker_2(dept, ids):
    Extension, Education = [], []

    for i in ids:
        soup = BeautifulSoup(requests.get("http://lhr.nu.edu.pk/"+dept+"/facultyProfile/"+str(i)).content,'html.parser')
        val = soup.find('div',class_='col-lg-8').find('ul')
        
        #error condition in case education details are unavailable
        if len(val.contents) > 1:
            Education.append(val.contents[1].text.strip())
        else:
            Education.append("")
        Extension.append(soup.find('div',class_='card').find('p',class_='mt-0').text.strip())
        
    return pd.DataFrame({
        'ID':ids,
        'Extension':Extension,
        'Education':Education
    })

#saving the dataframes of all departments as csv
[dataframe_maker_2(dept[i],ids_list[i]).to_csv(dept[i]+'_2.csv') for i in range(len(dept))]


[None, None, None, None, None]

In [12]:
#   -   -   -   -   -   -   Step-3   -   -   -   -   -   -   -

#fetching the general data of all teachers of all departments
samples = []
[samples.append(pd.read_csv(dept[i]+'.csv')) for i in range(len(dept))]

#fetching the specified data of all teachers of all departments
samples_2 = []
[samples_2.append(pd.read_csv(dept[i]+'_2.csv')) for i in range(len(dept))]

#merging the general and specified data of all teachers of all departments
merged_samples = []
[merged_samples.append(pd.merge(samples[i].drop(samples[i].columns[0], axis =1), samples_2[i].drop(samples_2[i].columns[0], axis =1), on='ID', how='left')) for i in range(len(dept))]

#concatenating data and saving as csv
pd.concat(merged_samples).reset_index().to_csv("fast_lhr_faculty.csv")

,ID,name,designation,HEC Approved PhD Supervisor,email,facultyURL,image,Extension,Education
0,1238,Dr. Kashif Zafar,Professor & HOD,True,kashif.zafar@nu.edu.pk,http://lhr.nu.edu.pk//fsc/facultyProfile/1238,http://lhr.nu.edu.pk//media/Faculty/kz1.jpeg,NaN,NaN
1,4391,Dr. Asif Mahmood Gillani,Professor,False,asif.gilani@nu.edu.pk,http://lhr.nu.edu.pk//fsc/facultyProfile/4391,http://lhr.nu.edu.pk//media/Faculty/03_Dr._Asi...,NaN,NaN
2,5424,Dr. Hammad Naveed,Professor,True,hammad.naveed@nu.edu.pk,http://lhr.nu.edu.pk//fsc/facultyProfile/5424,http://lhr.nu.edu.pk//media/Faculty/5424-remov...,NaN,NaN
3,6174,Dr. Arshad Ali,Associate Professor,True,arshad.ali1@nu.edu.pk,http://lhr.nu.edu.pk//fsc/facultyProfile/6174,http://lhr.nu.edu.pk//media/Faculty/5_Arshad_A...,NaN,NaN
4,9538,Dr. Irfan Younas,Associate Professor,True,irfan.younas@nu.edu.pk,http://lhr.nu.edu.pk//fsc/facultyProfile/9538,http://lhr.nu.edu.pk//media/Faculty/007_Dr._Ir...,NaN,NaN
...,...,...,...,...,...,...,...,...,...
67,6253,Ms. Mamoona Akbar Khan,Junior Lecturer,False,mamoona.akbar@nu.edu.pk,http://lhr.nu.edu.pk//fsc/facultyProfile/6253,http://lhr.nu.edu.pk//media/Faculty/6253-remov...,NaN,NaN
68,6554,Ms. Namra Absar,Lecturer (CS),False,namra.absar@nu.edu.pk,http://lhr.nu.edu.pk//fsc/facultyProfile/6554,http://lhr.nu.edu.pk//media/Faculty/6554.jpg,NaN,NaN
69,6176,Mr. Nauman Moazzam Hayat,Adj. Faculty,False,nauman.moazzam@nu.edu.pk,http://lhr.nu.edu.pk//fsc/facultyProfile/6176,http://lhr.nu.edu.pk//media/Faculty/6176-remov...,NaN,NaN
70,6552,Ms. Rubab Anam Janjua,Lecturer (CS),False,rubab.anam@nu.edu.pk,http://lhr.nu.edu.pk//fsc/facultyProfile/6552,http://lhr.nu.edu.pk//media/Faculty/6552.jpg,NaN,NaN


done
